Write a python code to launch a video file from google drive, then open it, run it from the 11th second, make snapshots once every second and save the images as PNG files in the 'output_images' directory, resize each image to 640x640 pixels

To achieve this task, you can use the following Python code. This code will:

Download a video file from Google Drive.
Open and play the video starting from the 11th second.
Capture snapshots every second and save them as PNG files in the 'output_images' directory.
Resize each image to 640x640 pixels.
Before running this code, make sure you have the necessary libraries installed. You can install them using pip install pydrive2 opencv-python pillow.

In [ ]:
import os
from io import BytesIO
import cv2
import time
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from PIL import Image
!pip install opencv-python

import gdown  # Import gdown library
from PIL import Image
from glob import glob
import hashlib
import shutil
import requests
import numpy as np
import gdown

In [ ]:
# Google Drive file URL (make sure it's publicly accessible)
file_url = 'https://drive.google.com/file/d/1G5UE0ntpPepY1fi2PHR082CbbffQH1ZG/view?usp=sharing'



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


write a code in python to extract files from a video from file_url = 'https://drive.google.com/file/d/1epp3I2VnmRrwG-ZewUdgijoM4jGxcTp9/view?usp=sharing' at a rate of 1 snapshot per second then save the images to one folder in PNG format. Then copy these images to the second folder but identify similar images and copy only the first image from the streaks of the similar images. Then split the images from the second folder into multiple images of the size 640 * 640 pixels and save these into the third folder.Use gdown to download the video.

In [ ]:
# Function to extract frames from a video and save them to a folder
def extract_frames_from_video(video_path, output_folder, frame_rate=1):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    frame_number = 0
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # Extract frames at the specified frame rate
        if frame_number % frame_rate == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frame_number}.png")
            cv2.imwrite(frame_filename, frame)

        frame_number += 1

    video_capture.release()

# Function to calculate an image hash
def calculate_image_hash(image_path):
    image = Image.open(image_path)
    image_hash = hashlib.md5(np.array(image)).hexdigest()
    return image_hash

# Function to copy unique frames to a destination folder
def copy_unique_frames(source_folder, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Create a dictionary to store image hashes and their corresponding file paths
    image_hashes = {}

    for root, _, files in os.walk(source_folder):
        for file in files:
            file_path = os.path.join(root, file)

            # Calculate the hash of the image
            hash_value = calculate_image_hash(file_path)

            # Check if the hash is already in the dictionary (indicating a duplicate)
            if hash_value not in image_hashes:
                image_hashes[hash_value] = file_path

    for hash_value, file_path in image_hashes.items():
        destination_path = os.path.join(destination_folder, os.path.basename(file_path))
        shutil.copy(file_path, destination_path)

# Function to split images into 640x640 segments and save them
def split_images(input_folder, output_folder, target_size=(640, 640)):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, _, files in os.walk(input_folder):
        for file in files:
            file_path = os.path.join(root, file)

            # Read the image
            image = cv2.imread(file_path)

            # Get the image dimensions
            height, width, _ = image.shape

            # Calculate the number of rows and columns for splitting
            rows = height // target_size[0]
            cols = width // target_size[1]

            for i in range(rows):
                for j in range(cols):
                    # Calculate the cropping coordinates
                    y1 = i * target_size[0]
                    y2 = (i + 1) * target_size[0]
                    x1 = j * target_size[1]
                    x2 = (j + 1) * target_size[1]

                    # Crop the image segment
                    segment = image[y1:y2, x1:x2]

                    # Save the cropped segment
                    segment_filename = os.path.join(output_folder, f"{file}_segment_{i}_{j}.png")
                    cv2.imwrite(segment_filename, segment)

if __name__ == "__main__":
    # Define folder paths, the part in the Google drive shared link between /d/ and /view?
    file_url = 'https://drive.google.com/file/d/1G5UE0ntpPepY1fi2PHR082CbbffQH1ZG/view?usp=sharing'
    first_folder = "first_folder"
    second_folder = "second_folder"
    third_folder = "third_folder"

    # Download the video file using gdown
    gdown.download(file_url, "downloaded_video.mp4", quiet=False)

    # Step 1: Extract frames from the video
    extract_frames_from_video("downloaded_video.mp4", first_folder)

    # Step 2: Copy unique frames to the second folder
    copy_unique_frames(first_folder, second_folder)

    # Step 3: Split images in the second folder into 640x640 segments and save to the third folder
    split_images(second_folder, third_folder, target_size=(640, 640))


Downloading...
From: https://drive.google.com/uc?id=1epp3I2VnmRrwG-ZewUdgijoM4jGxcTp9
To: /content/downloaded_video.mp4
100%|██████████| 218M/218M [00:07<00:00, 27.8MB/s]


Here's how the algorithm works:


1.   It defines a function calculate_image_hash that computes an MD5 hash of the grayscale version of an image.
2.  It defines a function find_duplicate_images that searches for duplicate images in a specified folder. It iterates through all the files in the folder, calculates the hash for each image, and checks if the hash already exists in a dictionary. If it does, the file is considered a duplicate and is deleted. Otherwise, the hash and file path are stored in the dictionary.

3. In the main part of the code, replace "path_to_your_folder_with_images" with the actual path to the folder containing your images.


4. When you run the script, it will identify and delete duplicate images in the specified folder, keeping only the first one encountered in each streak of duplicates.

Please be cautious when using this script, and make sure to back up your images before running it, as it will permanently delete duplicate images.
